In [9]:
# 权重衰减（weight decay）:应对过拟合问题的常用方法,等价于L2范数正则化（regularization）
# 正则化通过为模型损失函数添加惩罚项使学出的模型参数值较小，是应对过拟合的常用手段
# 以高维线性回归为例来引入一个过拟合问题,并使用权重衰减来应对过拟合
import tensorflow as tf
from tensorflow.keras import layers, models, initializers, optimizers, regularizers
import numpy as np
import matplotlib.pyplot as plt

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [4]:
tf.__version__

NameError: name 'tf' is not defined

In [4]:
!python --version

Python 3.11.4


In [5]:
n_train, n_test, num_inputs = 20, 100, 200
true_w, true_b = tf.ones((num_inputs, 1)) * 0.01, 0.05

In [6]:
features = tf.random.normal(shape=(n_train + n_test, num_inputs))
labels = tf.keras.backend.dot(features, true_w) + true_b
labels += tf.random.normal(mean=0.01, shape=labels.shape)
train_features, test_features = features[:n_train, :], features[n_train:, :]
train_labels, test_labels = labels[:n_train], labels[n_train:]

In [7]:
# 从零开始实现权重衰减的方法
# 定义随机初始化模型参数的函数。该函数为每个参数都附上梯度
def init_params():
    w = tf.Variable(tf.random.normal(mean=1, shape=(num_inputs, 1)))
    b = tf.Variable(tf.zeros(shape=(1,)))
    return [w, b]

In [15]:
# 定义L2范数惩罚项
def l2_penalty(w):
    return tf.reduce_sum((w**2)) / 2

In [24]:
import d2lzh_tensorflow2 as d2l

ModuleNotFoundError: No module named 'd2lzh_tensorflow2'

In [21]:
# 定义如何在训练数据集和测试数据集上分别训练和测试模型
batch_size, num_epochs, lr = 1, 100, 0.003
net, loss = d2l.linreg, d2l.squared_loss
optimizer = tf.keras.optimizers.SGD()
train_iter = tf.data.Dataset.from_tensor_slices(
    (train_features, train_labels)).batch(batch_size).shuffle(batch_size)

def fit_and_plot(lambd):
    w, b = init_params()
    train_ls, test_ls = [], []
    for _ in range(num_epochs):
        for X, y in train_iter:
            with tf.GradientTape(persistent=True) as tape:
                # 添加了L2范数惩罚项
                l = loss(net(X, w, b), y) + lambd * l2_penalty(w)
            grads = tape.gradient(l, [w, b])
            d2l.sgd([w, b], lr, batch_size, grads)
        train_ls.append(tf.reduce_mean(loss(net(train_features, w, b),
                             train_labels)).numpy())
        test_ls.append(tf.reduce_mean(loss(net(test_features, w, b),
                            test_labels)).numpy())
    d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'loss',
                 range(1, num_epochs + 1), test_ls, ['train', 'test'])
    print('L2 norm of w:', tf.norm(w).numpy())


AttributeError: module 'd2l' has no attribute 'linreg'